# Solving orbital equations with different algorithms

This notebook was adapted from `Orbit_games.ipynb`.



## Problem 2

This notebook is intended to include everything I have done for problem 2.

It includes:

- Part a): Two body motion for earth and sun
- Part b): Two body motion for heavy mass and light mass
- Part c): animations for both situations 
- Part d): RK23 vs Leapfrog methods 
- Part e): Three body motion**

## Formula

In two body system, the kinetic energy and potential energy are given as following:

$\begin{align}
    T = \frac12 m_1 (\dot x_1^2 + \dot y_1^2) + \frac12 m_2 (\dot x_2^2 + \dot y_2^2)
\end{align}$

$\begin{align}
    U = -\frac{G m_1 m_2}{r_{12}}
\end{align}$

Then we can calculate our favorite Lagrangian $\mathcal{L}$:

$\begin{align}
    \mathcal{L} &= T - U \\
                &= \frac12 m_1 (\dot x_1^2 + \dot y_1^2) + \frac12 m_2 (\dot x_2^2 + \dot y_2^2) + \frac{G m_1 m_2}{r_{12}}
\end{align}$

After that based on Euler-Lagrange equation

$\begin{align}
 \frac{d}{dt}\frac{\partial\mathcal{L}}{\partial \dot\phi} = \frac{\partial\mathcal L}{\partial\phi}
\end{align}$

We have can solve for time derivative for x and y for each body and construct a z-vector:

$\begin{align}
    \mathbf{z} = \left(\begin{array}{c} x_1 (t) \\ \dot x_1(t) \\ y_1(t)\\ \dot y_1(t) \\ x_2 (t) \\ \dot x_2(t) \\ y_2(t)\\ \dot y_2(t)  \end{array} \right) 
\end{align}$

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.integrate import solve_ivp

In [ ]:
# Change the common font size
font_size = 14
plt.rcParams.update({'font.size': font_size})

In [ ]:
class GravitationalOrbit:
    """
    This class uses Lagragian's equation for two-body orbiting due to gravitational force
    """
    
    def __init__(self, m_1=1., m_2=1., G=1.):
        self.m_1 = m_1
        self.m_2 = m_2
        self.G = G
        
    def dz_dt(self, t, z):
        """
        This function returns the right hand side of the differential equations
        t : float
            time
        z : float
            z[0] = x_1(t) z[1] = x_dot_1(t)
            z[2] = y_1(t) z[3] = y_dot_1(t)
            z[4] = x_2(t) z[5] = x_dot_2(t)
            z[6] = y_2(t) z[7] = y_dot_2(t)
            
        """
    
        r_12 = np.sqrt( (z[0] - z[4])**2 + (z[2] - z[6])**2 ) # distance between two bodies
        return [ \
                z[1], self.G * self.m_2 *(z[4] - z[0]) / r_12**3, \
                z[3], self.G * self.m_2 *(z[6] - z[2]) / r_12**3, \
                z[5], -self.G * self.m_1 *(z[4] - z[0]) / r_12**3, \
                z[7], -self.G * self.m_1 *(z[6] - z[2]) / r_12**3, \
               ]
        
    
    def solve_ode(self, t_pts, z_0, abserr=1.0e-8, relerr=1.0e-8):
        """
        Solve the ODE given initial conditions.
        Use solve_ivp with the option of specifying the method.
        Specify smaller abserr and relerr to get more precision.
        """

        solution = solve_ivp(self.dz_dt, (t_pts[0], t_pts[-1]), 
                             z_0, t_eval=t_pts, method='RK23', 
                             atol=abserr, rtol=relerr)
        x_1, x_dot_1, y_1, y_dot_1, x_2, x_dot_2, y_2, y_dot_2 = solution.y
        return x_1, x_dot_1, y_1, y_dot_1, x_2, x_dot_2, y_2, y_dot_2
    

    def solve_ode_Leapfrog(self, t_pts, z_0):
        """
        Solve the ODE given initial conditions with the Leapfrog method.
        """
        delta_t = t_pts[1] - t_pts[0]
        
        x_1_0, x_dot_1_0, y_1_0, y_dot_1_0, x_2_0, x_dot_2_0, y_2_0, y_dot_2_0 = z_0
        
        # initialize the arrays for x, x_dot, y, y_dot
        num_t_pts = len(t_pts)
        
        x_1 = np.zeros(num_t_pts)
        x_dot_1 = np.zeros(num_t_pts)
        x_dot_1_half = np.zeros(num_t_pts)
        
        y_1 = np.zeros(num_t_pts)
        y_dot_1 = np.zeros(num_t_pts)
        y_dot_1_half = np.zeros(num_t_pts)

        x_2 = np.zeros(num_t_pts)
        x_dot_2 = np.zeros(num_t_pts)
        x_dot_2_half = np.zeros(num_t_pts)

        y_2 = np.zeros(num_t_pts)
        y_dot_2 = np.zeros(num_t_pts)
        y_dot_2_half = np.zeros(num_t_pts)


        
        # initial conditions
        x_1[0] = x_1_0
        x_dot_1[0] = x_dot_1_0

        y_1[0] = y_1_0
        y_dot_1[0] = y_dot_1_0
        
        x_2[0] = x_2_0
        x_dot_2[0] = x_dot_2_0
        
        y_2[0] = y_2_0
        y_dot_2[0] = y_dot_2_0
        
        # step through the differential equation
        for i in np.arange(num_t_pts - 1):
            t = t_pts[i]
            
            z = [x_1[i], x_dot_1[i], y_1[i], y_dot_1[i], \
                 x_2[i], x_dot_2[i], y_2[i], y_dot_2[i]]
            out = self.dz_dt(t,z)
            
            x_dot_1_half[i] = x_dot_1[i] + out[1] * delta_t/2.
            x_1[i+1] = x_1[i] + x_dot_1_half[i] * delta_t
            
            y_dot_1_half[i] = y_dot_1[i] + out[3] * delta_t/2.
            y_1[i+1] = y_1[i] + y_dot_1_half[i] * delta_t
            
            x_dot_2_half[i] = x_dot_2[i] + out[5] * delta_t/2.
            x_2[i+1] = x_2[i] + x_dot_2_half[i] * delta_t
            
            y_dot_2_half[i] = y_dot_2[i] + out[7] * delta_t/2.
            y_2[i+1] = y_2[i] + y_dot_2_half[i] * delta_t
            
            z = [x_1[i+1], x_dot_1[i], y_1[i+1], y_dot_1[i], \
                 x_2[i+1], x_dot_2[i], y_2[i+1], y_dot_2[i]]
            out = self.dz_dt(t,z)
            
        return x_1, x_dot_1, y_1, y_dot_1, x_2, x_dot_2, y_2, y_dot_2
    
    def energy(self, t_pts, x_1, x_dot_1, y_1, y_dot_1, x_2, x_dot_2, y_2, y_dot_2):
        
        r_12 = np.sqrt((x_1 - x_2)**2 + (y_1 - y_2)**2 )
    
        return (self.m_1/2.) * (x_dot_1**2 + y_dot_1**2) + (self.m_2/2.) * (x_dot_2**2 + y_dot_2**2) \
                - self.G * m_1 * m_2 / r_12

In [ ]:
def plot_y_vs_x(x, y, axis_labels=None, label=None, title=None, 
                color=None, linestyle=None, semilogy=False, loglog=False,
                ax=None):
    """
    Generic plotting function: return a figure axis with a plot of y vs. x,
    with line color and style, title, axis labels, and line label
    """
    if ax is None:        # if the axis object doesn't exist, make one
        ax = plt.gca()

    if (semilogy):
        line, = ax.semilogy(x, y, label=label, 
                            color=color, linestyle=linestyle)
    elif (loglog):
        line, = ax.loglog(x, y, label=label, 
                          color=color, linestyle=linestyle)
    else:
        line, = ax.plot(x, y, label=label, 
                    color=color, linestyle=linestyle)

    if label is not None:    # if a label if passed, show the legend
        ax.legend()
    if title is not None:    # set a title if one if passed
        ax.set_title(title)
    if axis_labels is not None:  # set x-axis and y-axis labels if passed  
        ax.set_xlabel(axis_labels[0])
        ax.set_ylabel(axis_labels[1])

    return ax, line

In [ ]:
def start_stop_indices(t_pts, plot_start, plot_stop):
    start_index = (np.fabs(t_pts-plot_start)).argmin()  # index in t_pts array 
    stop_index = (np.fabs(t_pts-plot_stop)).argmin()  # index in t_pts array 
    return start_index, stop_index

## Part a) Two-body (Earth and sun)

In this part, two masses ($m1 = 1$ standing for earth, and $m2 = 5$ standing for sun) are simulated. 

In [ ]:
# Label for plots
orbit_label = (r'$x$', r'$y$')

In [ ]:
# Plotting time 
t_start = 0.
t_end = 10.
delta_t = 0.01

t_pts = np.arange(t_start, t_end+delta_t, delta_t)  

# Initial conditions
G = 1.
m_1 = 1.
m_2 = 5.

# Instantiate the class
o1 = GravitationalOrbit(m_1, m_2, G)

# Initial conditions with COM at rest
x_1_0, x_dot_1_0 = 1., -1.
y_1_0, y_dot_1_0 = 1., 1.
x_2_0, x_dot_2_0 = -(m_1 / m_2) * x_1_0, -(m_1 / m_2) * x_dot_1_0
y_2_0, y_dot_2_0 = -(m_1 / m_2) * y_1_0, -(m_1 / m_2) * y_dot_1_0

z_0 = [x_1_0, x_dot_1_0, y_1_0, y_dot_1_0, \
       x_2_0, x_dot_2_0, y_2_0, y_dot_2_0]

x_1, x_dot_1, y_1, y_dot_1, x_2, x_dot_2, y_2, y_dot_2 = o1.solve_ode(t_pts, z_0)

In [ ]:
fig = plt.figure(figsize=(5,5))


# Start the plot

ax = fig.add_subplot(1,1,1)
start, stop = start_stop_indices(t_pts, t_start, t_end)
ax.plot(x_1, y_1, color='blue', label=r'$m_1$')
ax.plot(x_2, y_2, color='red', label=r'$m_2$')
ax.set_title('Simple gravitational orbit')
ax.legend()
ax.set_aspect(1)


fig.tight_layout()
fig.savefig('simple_orbit.png', dpi=200, bbox_inches='tight')



## Part b) Two-body (with very heavy planet)

In this part, two masses ($m1 = 1$ standing for light body, and $m2 = 50$ standing for heavy body) are simulated. 

Based on the simulation below, we can see that as the heavy body $m2$ >> $m1$, $m2$ is essentially at rest while $m1$, the light body, is orbiting in a elliptical way.

In [ ]:
# Label for plots
orbit_label = (r'$x$', r'$y$')

# Plotting time 
t_start = 0.
t_end = 10.
delta_t = 0.01

t_pts = np.arange(t_start, t_end+delta_t, delta_t)  

# Initial conditions
G = 1.
m_1 = 1.
m_2 = 50.

# Instantiate the class
o1 = GravitationalOrbit(m_1, m_2, G)

# Initial conditions with COM at rest
x_1_0, x_dot_1_0 = 1., -1.
y_1_0, y_dot_1_0 = 1., 1.
x_2_0, x_dot_2_0 = -(m_1 / m_2) * x_1_0, -(m_1 / m_2) * x_dot_1_0
y_2_0, y_dot_2_0 = -(m_1 / m_2) * y_1_0, -(m_1 / m_2) * y_dot_1_0

z_0 = [x_1_0, x_dot_1_0, y_1_0, y_dot_1_0, \
       x_2_0, x_dot_2_0, y_2_0, y_dot_2_0]

x_1b, x_dot_1b, y_1b, y_dot_1b, x_2b, x_dot_2b, y_2b, y_dot_2b = o1.solve_ode(t_pts, z_0)

fig = plt.figure(figsize=(5,5))


# Start the plot

ax = fig.add_subplot(1,1,1)
start, stop = start_stop_indices(t_pts, t_start, t_end)
ax.plot(x_1b, y_1b, color='blue', label=r'$m_1$')
ax.plot(x_2b, y_2b, color='red', label=r'$m_2$')
ax.set_title('Simple gravitational orbit')
ax.legend()
ax.set_aspect(1)


fig.tight_layout()
fig.savefig('heavy_simple_orbit.png', dpi=200, bbox_inches='tight')


## Part c) Two-body Animation


In [ ]:
from matplotlib import animation
from IPython.display import HTML
import matplotlib
matplotlib.rcParams['animation.embed_limit'] = 2**128


In [ ]:
%%capture

x_min = -1.5
x_max = -x_min
y_min = -1.5
y_max = -y_min

fig_anim = plt.figure(figsize=(10,5), num='Two Body Orbiting')
ax_anim = fig_anim.add_subplot(1,1,1)  
ax_anim.set_xlim(x_min, x_max)
ax_anim.set_ylim(y_min, y_max)

ln1_anim, = ax_anim.plot(x_1, y_1, color='blue', lw=1)
ln2_anim, = ax_anim.plot(x_2, y_2, color='red', lw=1)

pt1_anim, = ax_anim.plot(x_1[0], y_1[0], 'o', markersize=8, color='blue')
pt2_anim, = ax_anim.plot(x_2[0], y_2[0], 'o', markersize=8, color='red')

ax_anim.set_aspect(1)
ax_anim.axis('off')
fig_anim.tight_layout()

def animate_orbits(i):
    i_skip = 1 * i
    
    pt1_anim.set_data(x_1[i_skip], y_1[i_skip])
    pt2_anim.set_data(x_2[i_skip], y_2[i_skip])
    
    return (pt1_anim, pt2_anim)

frame_interval = 1.  # time between frames
frame_number = 1001   # number of frames to include (index of t_pts)
anim = animation.FuncAnimation(fig_anim,
                               animate_orbits,
                               init_func=None,
                               frames=frame_number, 
                               interval=frame_interval, 
                               blit=True,
                               repeat=False)

In [ ]:
HTML(anim.to_jshtml())  # animate using javascript

In [ ]:
%%capture

x_min = -1.2
x_max = -x_min
y_min = -1.2
y_max = -y_min

fig_anim = plt.figure(figsize=(10,5), num='Two Body Orbiting')
ax_anim = fig_anim.add_subplot(1,1,1)  
ax_anim.set_xlim(x_min, x_max)
ax_anim.set_ylim(y_min, y_max)

ln1_anim, = ax_anim.plot(x_1b, y_1b, color='blue', lw=1)
ln2_anim, = ax_anim.plot(x_2b, y_2b, color='red', lw=1)

pt1_anim, = ax_anim.plot(x_1b[0], y_1b[0], 'o', markersize=8, color='blue')
pt2_anim, = ax_anim.plot(x_2b[0], y_2b[0], 'o', markersize=8, color='red')

ax_anim.set_aspect(1)
ax_anim.axis('off')
fig_anim.tight_layout()

def animate_orbits(i):
    i_skip = 1 * i
    
    pt1_anim.set_data(x_1b[i_skip], y_1b[i_skip])
    pt2_anim.set_data(x_2b[i_skip], y_2b[i_skip])
    
    return (pt1_anim, pt2_anim)

frame_interval = 1.  # time between frames
frame_number = 1001   # number of frames to include (index of t_pts)
anim = animation.FuncAnimation(fig_anim,
                               animate_orbits,
                               init_func=None,
                               frames=frame_number, 
                               interval=frame_interval, 
                               blit=True,
                               repeat=False)

In [ ]:
HTML(anim.to_jshtml())  # animate using javascript

## Part d) Leap-frog


In [ ]:
# Plotting time 
t_start = 0.
t_end = 100.
delta_t = 0.01

t_pts = np.arange(t_start, t_end+delta_t, delta_t)  

# Initial conditions
G = 1.
m_1 = 1.
m_2 = 5.

# Instantiate the class
o1 = GravitationalOrbit(m_1, m_2, G)

# Initial conditions with COM at rest
x_1_0, x_dot_1_0 = 1., -1.
y_1_0, y_dot_1_0 = 1., 1.
x_2_0, x_dot_2_0 = -(m_1 / m_2) * x_1_0, -(m_1 / m_2) * x_dot_1_0
y_2_0, y_dot_2_0 = -(m_1 / m_2) * y_1_0, -(m_1 / m_2) * y_dot_1_0

z_0 = [x_1_0, x_dot_1_0, y_1_0, y_dot_1_0, \
       x_2_0, x_dot_2_0, y_2_0, y_dot_2_0]

x_1, x_dot_1, y_1, y_dot_1, x_2, x_dot_2, y_2, y_dot_2 = o1.solve_ode(t_pts, z_0)
x_1_lp, x_dot_1_lp, y_1_lp, y_dot_1_lp, x_2_lp, x_dot_2_lp, y_2_lp, y_dot_2_lp = o1.solve_ode_Leapfrog(t_pts, z_0)

E_tot_pts = o1.energy(t_pts, x_1, x_dot_1, y_1, y_dot_1, x_2, x_dot_2, y_2, y_dot_2)
E_tot_0 = E_tot_pts[0]
E_tot_rel_pts = np.abs((E_tot_pts - E_tot_0)/E_tot_0)

E_tot_pts_LF = o1.energy(t_pts, x_1_lp, x_dot_1_lp, y_1_lp, y_dot_1_lp, x_2_lp, x_dot_2_lp, y_2_lp, y_dot_2_lp)
E_tot_0_LF = E_tot_pts_LF[0]
E_tot_rel_pts_LF = np.abs((E_tot_pts_LF - E_tot_0_LF)/E_tot_0_LF)

In [ ]:
fig_energydiff = plt.figure(figsize=(6,6))

ax_5a = fig_energydiff.add_subplot(1,1,1)
#ax_5a.semilogy(t_pts, np.abs(E_tot_pts), color='black', label=r'$E(t)$')
ax_5a.semilogy(t_pts, E_tot_rel_pts, 
               color='green', label=r'$\Delta E(t)$ RK23')
ax_5a.semilogy(t_pts, E_tot_rel_pts_LF, 
               color='red', label=r'$\Delta E(t)$ Leapfrog')
ax_5a.set_ylim(1.e-10, 1.e-2)    # (1.e-12, 5)
ax_5a.set_xlabel(r'$t$')
ax_5a.set_ylabel(r'Energy')
ax_5a.set_title('Change in energy with time')
ax_5a.legend()

fig_energydiff.tight_layout()


## Part e) Three-body (equal masses)

In [ ]:
class GravitationalOrbit3:
    """
    This class uses Lagragian's equation for two-body orbiting due to gravitational force
    """
    
    def __init__(self, m_1=1., m_2=1., m_3=1, G=1.):
        self.m_1 = m_1
        self.m_2 = m_2
        self.m_3 = m_3
        self.G = G
        
    def dz_dt(self, t, z):
        """
        This function returns the right hand side of the differential equations
        t : float
            time
        z : float
            z[0] = x_1(t) z[1] = x_dot_1(t)
            z[2] = y_1(t) z[3] = y_dot_1(t)
            z[4] = x_2(t) z[5] = x_dot_2(t)
            z[6] = y_2(t) z[7] = y_dot_2(t)
            z[8] = x_3(t) z[9] = x_dot_3(t)
            z[10] = y_3(t) z[11] = y_dot_3(t)
            
        """
    
        r_12 = np.sqrt( (z[0] - z[4])**2 + (z[2] - z[6])**2 ) # distance between two bodies
        r_13 = np.sqrt( (z[0] - z[8])**2 + (z[2] - z[10])**2 )
        r_23 = np.sqrt( (z[4] - z[8])**2 + (z[4] - z[10])**2 )
        return [
                z[1], 
                self.G*self.m_2*(z[4]-z[0])/(r_12)**(3/2)+self.G*self.m_3*(z[8]-z[0])/(r_13)**(3/2),
                z[3], 
                self.G*self.m_2*(z[6]-z[2])/(r_12)**(3/2)+self.G*self.m_3*(z[10]-z[2])/(r_13)**(3/2),
                z[5], 
                self.G*self.m_1*(z[0]-z[4])/(r_12)**(3/2)+self.G*self.m_3*(z[8]-z[4])/(r_23)**(3/2),
                z[7], 
                self.G*self.m_1*(z[2]-z[6])/(r_12)**(3/2)+self.G*self.m_3*(z[10]-z[6])/(r_23)**(3/2),
                z[9], 
                self.G*self.m_1*(z[0]-z[8])/(r_13)**(3/2)+self.G*self.m_2*(z[4]-z[8])/(r_23)**(3/2),
                z[11], 
                self.G*self.m_1*(z[2]-z[10])/(r_13)**(3/2)+self.G*self.m_2*(z[6]-z[10])/(r_23)**(3/2),
               ]
        
    
    def solve_ode(self, t_pts, z_0, abserr=1.0e-8, relerr=1.0e-8):
        """
        Solve the ODE given initial conditions.
        Use solve_ivp with the option of specifying the method.
        Specify smaller abserr and relerr to get more precision.
        """

        solution = solve_ivp(self.dz_dt, (t_pts[0], t_pts[-1]), 
                             z_0, t_eval=t_pts, method='RK23', 
                             atol=abserr, rtol=relerr)
        x_1, x_dot_1, y_1, y_dot_1, x_2, x_dot_2, y_2, y_dot_2, x_3, x_dot_3, y_3, y_dot_3 = solution.y
        return x_1, x_dot_1, y_1, y_dot_1, x_2, x_dot_2, y_2, y_dot_2, x_3, x_dot_3, y_3, y_dot_3

In [ ]:
# Plotting time 
t_start = 0.
t_end = 100.
delta_t = 0.01

t_pts = np.arange(t_start, t_end+delta_t, delta_t)  

# Initial conditions
G = 1.
m_1 = 5.
m_2 = 1.
m_3 = 1.

# Instantiate the class
o2 = GravitationalOrbit3(m_1, m_2, m_3, G)

# Initial conditions found online

x_3_0, x_dot_3_0 = 0., -.93240737
y_3_0, y_dot_3_0 = 0., -.86473146 
x_1_0, x_dot_1_0 = 0.97000436, -x_dot_3_0/2.
y_1_0, y_dot_1_0 = -.24308753, -y_dot_3_0/2.
x_2_0, x_dot_2_0 = - x_1_0, -x_dot_3_0/2.
y_2_0, y_dot_2_0 = - y_1_0, -y_dot_3_0/2.
  

z_0 = [x_1_0, x_dot_1_0, y_1_0, y_dot_1_0, \
       x_2_0, x_dot_2_0, y_2_0, y_dot_2_0, \
       x_3_0, x_dot_3_0, y_3_0, y_dot_3_0]

x_1, x_dot_1, y_1, y_dot_1, x_2, x_dot_2, y_2, y_dot_2, x_3, x_dot_3, y_3, y_dot_3 = o2.solve_ode(t_pts, z_0)

In [ ]:
fig = plt.figure(figsize=(5,5))


# Start the plot

ax = fig.add_subplot(1,1,1)
start, stop = start_stop_indices(t_pts, t_start, t_end)
ax.plot(x_1, y_1, color='blue', label=r'$m_1$')
ax.plot(x_2, y_2, color='red', label=r'$m_2$')
ax.plot(x_3, y_3, color='green', label=r'$m_3$')
ax.set_title('Three Body Motion')
ax.legend()
ax.set_aspect('equal')
#ax.set_xlim(-60,60)

fig.tight_layout()
